In [1]:
import os
import pandas as pd

# Set the working directory
os.chdir("C:\TradeGroup")
my_wd = os.getcwd()

file_path_Exp_Others = f"{my_wd}\\Data\\Mirror_Data\\Mirror-Others-GExReEx-"
file_path_Imp_Others   = f"{my_wd}\\Data\\Mirror_Data\\Mirror-Others-GImReIm-"

file_path_Exp_Iran = f"{my_wd}\\Data\\Mirror_Data\\Mirror-Iran-GExReEx-"
file_path_Imp_Iran   = f"{my_wd}\\Data\\Mirror_Data\\Mirror-Iran-GImReIm-"


def create_folder(folder_path):
    if not os.path.exists(f'{my_wd}/Data/{folder_path}'):
        os.makedirs(f'{my_wd}/Data/{folder_path}')
    else:
        print(f'Already Exists')

create_folder('Mirror_Data/Results')


Already Exists


In [52]:
for year in range(2017,2022):
    OExp=pd.read_csv(f"{file_path_Exp_Others}{year}.csv")
    IExp=pd.read_csv(f"{file_path_Exp_Iran}{year}.csv")
    OImp=pd.read_csv(f"{file_path_Imp_Others}{year}.csv")
    IImp=pd.read_csv(f"{file_path_Imp_Iran}{year}.csv")
    Summed=[]
    #List_of_Dfs=[OExp,IExp,OImp,IImp]
    List_of_Dfs = [('OExp', OExp), ('IExp', IExp), ('OImp', OImp), ('IImp', IImp)]
    for df_name, df in List_of_Dfs:
          #df['HS2']=df['ProductCode'].str[:2].str.zfill(2)
         #df['HS2']=df['ProductCode']
         #df=df[(df['HS2'].str.len()==3) or (df['HS2'].str.len()==4)]
         df=df[(df['ProductCode'].str.len()==3) | (df['ProductCode'].str.len()==4)]
         df['HS4']=df['ProductCode'].str[:4].str.zfill(4)
         df['HS2']=df['HS4'].str[0:2]

         Summations= df.groupby(['ReporterName','PartnerName','HS2','TradeFlowCode']).agg(
         ReporterName=('ReporterName',"first"),
         PartnerName=('PartnerName','first'),
         HS2=('HS2','first'),
         TradeFlowCode=('TradeFlowCode','first'),
         Trade=('TradeValue in 1000 USD','sum')
         ).reset_index(drop=True)
         globals()[df_name + '_Summations'] = Summations
         Summed.append((f'{df_name}',globals()[df_name + '_Summations']))
         xlsx_filename_Sum = os.path.join(my_wd, f'Data\\Mirror_Data\\Results\\{df_name}_{year}.xlsx')
         #Summations.to_excel(xlsx_filename_Sum, index=False)
    Modified_Sum=[]     
    for df_name2, dff in Summed:
      if df_name2=="OExp":
        New=dff.loc[:,['ReporterName','HS2','TradeFlowCode','Trade']]
        column_mapping = {'ReporterName': 'Country','TradeFlowCode':'Exp_type', 'Trade': 'Export_to_Iran'}
        
        # Rename columns using the dictionary
        New.rename(columns=column_mapping, inplace=True)
        
      if df_name2=='IImp':
        New=dff.loc[:,['PartnerName','HS2','TradeFlowCode','Trade']]
        column_mapping = {'PartnerName':'Country','TradeFlowCode':'Imp_type','Trade':'Import_to_Iran'}
        New.rename(columns=column_mapping, inplace=True)
        
      if df_name2=="IExp":
        New=dff.loc[:,['PartnerName','HS2','TradeFlowCode','Trade']]
        column_mapping = {'PartnerName': 'Country','TradeFlowCode':'Exp_type', 'Trade': 'Export_from_Iran'}
        New.rename(columns=column_mapping,inplace=True)

        
        
      if df_name2=="OImp":
        New=dff.loc[:,['ReporterName','HS2','TradeFlowCode','Trade']]
        column_mapping = {'ReporterName': 'Country','TradeFlowCode':'Imp_type', 'Trade': 'Import_from_Iran'}
        New.rename(columns=column_mapping,inplace=True)

        
      Modified_Sum.append(New)
      
    IranImpMirror=pd.merge(Modified_Sum[0],Modified_Sum[3],on=['Country','HS2'],how='outer')
    IranImpMirror['Import_to_Iran'] = IranImpMirror['Import_to_Iran'].fillna(0)
    IranImpMirror['Export_to_Iran'] = IranImpMirror['Export_to_Iran'].fillna(0)
    IranImpMirror['IFF_Imp']=IranImpMirror['Export_to_Iran']-IranImpMirror['Import_to_Iran']
    
    IranExpMirror=pd.merge(Modified_Sum[1],Modified_Sum[2],on=['Country','HS2'],how='outer')
    IranExpMirror['Import_from_Iran'] = IranExpMirror['Import_from_Iran'].fillna(0)
    IranExpMirror['Export_from_Iran'] = IranExpMirror['Export_from_Iran'].fillna(0)
    IranExpMirror['IFF_Exp']=IranExpMirror['Export_from_Iran']-IranExpMirror['Import_from_Iran']
    
    
    
    Mirror_Imp = os.path.join(my_wd, f'Data\\Mirror_Data\\Results\\Mirror_Imp_{year}.xlsx')
    IranImpMirror.to_excel(Mirror_Imp, index=False)
    
    Mirror_Exp = os.path.join(my_wd, f'Data\\Mirror_Data\\Results\\Mirror_Exp_{year}.xlsx')
    IranExpMirror.to_excel(Mirror_Exp, index=False)

    
        

    

C:\Users\msbirjandi\AppData\Local\Temp\ipykernel_15188\4007768447.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['HS4']=df['ProductCode'].str[:4].str.zfill(4)
C:\Users\msbirjandi\AppData\Local\Temp\ipykernel_15188\4007768447.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['HS2']=df['HS4'].str[0:2]
C:\Users\msbirjandi\AppData\Local\Temp\ipykernel_15188\4007768447.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [2]:
for year in range(2017,2022):
   ImpMirr=pd.read_excel(os.path.join(my_wd, f'Data\\Mirror_Data\\Results\\Mirror_Imp_{year}.xlsx'))
   ExpMirr=pd.read_excel(os.path.join(my_wd, f'Data\\Mirror_Data\\Results\\Mirror_Exp_{year}.xlsx'))


   ImpMirr = ImpMirr.groupby(['Country','HS2']).agg(
            IFF_Imp=("IFF_Imp", "sum")
        ).reset_index()
   
   ExpMirr = ExpMirr.groupby(['Country','HS2']).agg(
            IFF_Exp=("IFF_Exp", "sum")
        ).reset_index()
   

   ImpMirr['IFF_Imp'+'_'+str(year)]=ImpMirr['IFF_Imp']
   ExpMirr['IFF_Exp'+'_'+str(year)]=ExpMirr['IFF_Exp']
   




   YearIFFs=ImpMirr[['Country','HS2','IFF_Imp'+'_'+str(year)]].merge(ExpMirr[['Country','HS2','IFF_Exp'+'_'+str(year)]],
                                                               on=['Country','HS2'],how='outer')
   if year==2017:
       AllIIFs=YearIFFs
   else:
       AllIIFs=AllIIFs.merge(YearIFFs,on=['Country','HS2'],how='outer')
AllIIFs.fillna(0, inplace=True)       
Countries_IFFs=AllIIFs[(AllIIFs['Country']!=" World") & (AllIIFs['Country']!="European Union")]
World_IFFs=AllIIFs[(AllIIFs['Country']=="World")]

All_Countries_Path = os.path.join(my_wd, f'Data\\Mirror_Data\\Results\\All_Countries_IFFs.xlsx')
World_Path = os.path.join(my_wd, f'Data\\Mirror_Data\\Results\\World_IFFs.xlsx')

#Countries_IFFs.to_excel(All_Countries_Path,index=False)
#World_IFFs.to_excel(World_Path,index=False)

In [61]:
Countries_Sum=Countries_IFFs.groupby(['Country']).agg(
    IFF_Imp_2017=("IFF_Imp_2017","sum"),
    IFF_Imp_2018=("IFF_Imp_2018","sum"),
    IFF_Imp_2019=("IFF_Imp_2019","sum"),
    IFF_Imp_2020=("IFF_Imp_2020","sum"),
    IFF_Imp_2021=("IFF_Imp_2021","sum"),

    IFF_Exp_2017=("IFF_Exp_2017","sum"),
    IFF_Exp_2018=("IFF_Exp_2018","sum"),
    IFF_EXp_2019=("IFF_Exp_2019","sum"),
    IFF_Exp_2020=("IFF_Exp_2020","sum"),
    IFF_Exp_2021=("IFF_Exp_2021","sum"),

)



HS2_Sum=Countries_IFFs.groupby(['HS2']).agg(
    IFF_Imp_2017=("IFF_Imp_2017","sum"),
    IFF_Imp_2018=("IFF_Imp_2018","sum"),
    IFF_Imp_2019=("IFF_Imp_2019","sum"),
    IFF_Imp_2020=("IFF_Imp_2020","sum"),
    IFF_Imp_2021=("IFF_Imp_2021","sum"),

    IFF_Exp_2017=("IFF_Exp_2017","sum"),
    IFF_Exp_2018=("IFF_Exp_2018","sum"),
    IFF_EXp_2019=("IFF_Exp_2019","sum"),
    IFF_Exp_2020=("IFF_Exp_2020","sum"),
    IFF_Exp_2021=("IFF_Exp_2021","sum"),

)

Countries_Sum_Path = os.path.join(my_wd, f'Data\\Mirror_Data\\Results\\Countries_Sum.xlsx')
HS2_Sum_Path = os.path.join(my_wd, f'Data\\Mirror_Data\\Results\\HS2_Sum.xlsx')


Countries_Sum.to_excel(Countries_Sum_Path,index=True)
HS2_Sum.to_excel(HS2_Sum_Path,index=True)


In [81]:
bins=[0, 5,15,24,26,27,38,40,43,49,63,67,71,83,85,89,99]
values=["Animal",'Vegetable','FoodProd',"Minerals","Fuels",
        'Chemicals',"PlasticRub","HidesSkin",'Wood',
        'TextCloth','FootWear',"StonGlas","Metals",
        'MachElec',"Trnsport","Miscellan"]
Countries_IFFs['ProductType']=pd.cut(Countries_IFFs['HS2'],bins=bins,
                                     labels=values)
Countries_IFFs


Countries_Grouped_HS=Countries_IFFs.groupby(['Country','ProductType']).agg(
    IFF_Imp_2017=("IFF_Imp_2017","sum"),
    IFF_Imp_2018=("IFF_Imp_2018","sum"),
    IFF_Imp_2019=("IFF_Imp_2019","sum"),
    IFF_Imp_2020=("IFF_Imp_2020","sum"),
    IFF_Imp_2021=("IFF_Imp_2021","sum"),

    IFF_Exp_2017=("IFF_Exp_2017","sum"),
    IFF_Exp_2018=("IFF_Exp_2018","sum"),
    IFF_EXp_2019=("IFF_Exp_2019","sum"),
    IFF_Exp_2020=("IFF_Exp_2020","sum"),
    IFF_Exp_2021=("IFF_Exp_2021","sum"),

)

HS2_Grouped_Sum=Countries_IFFs.groupby(['ProductType']).agg(
    IFF_Imp_2017=("IFF_Imp_2017","sum"),
    IFF_Imp_2018=("IFF_Imp_2018","sum"),
    IFF_Imp_2019=("IFF_Imp_2019","sum"),
    IFF_Imp_2020=("IFF_Imp_2020","sum"),
    IFF_Imp_2021=("IFF_Imp_2021","sum"),

    IFF_Exp_2017=("IFF_Exp_2017","sum"),
    IFF_Exp_2018=("IFF_Exp_2018","sum"),
    IFF_EXp_2019=("IFF_Exp_2019","sum"),
    IFF_Exp_2020=("IFF_Exp_2020","sum"),
    IFF_Exp_2021=("IFF_Exp_2021","sum"),

)


Countries_Grouped_Sum_Path = os.path.join(my_wd, f'Data\\Mirror_Data\\Results\\Countries_Grouped_Sum.xlsx')
HS2_Grouped_Sum_Path = os.path.join(my_wd, f'Data\\Mirror_Data\\Results\\HS2_Grouped_Sum.xlsx')


Countries_Grouped_HS.to_excel(Countries_Grouped_Sum_Path,index=True)
HS2_Grouped_Sum.to_excel(HS2_Grouped_Sum_Path,index=True)

C:\Users\msbirjandi\AppData\Local\Temp\ipykernel_15188\4030557672.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Countries_IFFs['ProductType']=pd.cut(Countries_IFFs['HS2'],bins=bins,
C:\Users\msbirjandi\AppData\Local\Temp\ipykernel_15188\4030557672.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  Countries_Grouped_HS=Countries_IFFs.groupby(['Country','ProductType']).agg(
C:\Users\msbirjandi\AppData\Local\Temp\ipykernel_15188\4030557672.py:26: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version o